In [1]:
from os import listdir
from os.path import join, isdir

import numpy as np
from scipy.io import loadmat
from scipy import signal

import matplotlib.pyplot as plt
import pandas as pd
import mne
import mne_features


path_to_eeg = 'data/filtered_eeg'
path_to_feeltrace = '../../converted_data'
path_to_figures = './figures'
path_to_prediction_data = 'data/processed_features_and_labels/2'


Bad key "text.kerning_factor" on line 4 in
/Users/Qianqian/anaconda3/envs/ml/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
eeg_data = pd.read_csv(join(path_to_eeg, 'p2_eeg.csv'), delimiter=' ', header=None) * (10 ** 6) # turns Volts to microVolts

In [3]:
eeg_data.columns = ['E'+ str(i+1) for i in range(64)]
eeg_data

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,...,E55,E56,E57,E58,E59,E60,E61,E62,E63,E64
0,0.014176,0.005387,0.002584,0.000328,0.005121,0.000237,-0.000347,-0.000079,-0.001821,-0.003507,...,0.005199,0.005981,0.004800,0.008801,0.007403,0.004973,0.009077,1147.051514,-0.009534,-0.008903
1,-15.035622,-3.348845,-4.315351,-1.252903,-13.060442,-0.920893,-0.542530,-3.827675,-4.522109,-2.319304,...,-19.778339,35.043999,-2.587749,-4.922697,-4.188222,-6.676576,-15.021190,1133.590088,-14.521353,-29.384942
2,-3.847582,-6.294374,-2.460590,-0.228796,-9.929828,0.033820,-1.526018,-0.842323,-3.187743,3.889015,...,-16.525899,34.371235,-5.003324,0.604902,-5.521495,-6.924783,-2.707489,1148.293945,-5.210410,-11.462566
3,14.274408,-3.391993,4.342655,2.089066,7.434633,4.033899,-1.151987,7.388918,3.202800,12.598860,...,-6.540090,19.219042,-3.402330,7.945770,-0.712348,0.949606,17.775012,1154.135010,16.914823,22.378679
4,13.668109,2.340968,7.732204,2.814299,20.967508,7.686130,0.529111,12.052141,8.116241,14.166424,...,-12.922561,21.405678,-0.447362,6.039142,3.709986,7.645095,16.668056,1148.333740,25.214796,21.441393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732541,112.521284,105.075297,49.212512,-2.646478,100.190666,49.232114,16.424328,219.213712,16.893728,197.226101,...,-73.404827,124.353209,132.207041,3.383307,7.965502,99.130478,18.087732,0.000000,-11.593908,28.160725
732542,101.494471,94.194577,43.777850,-2.300054,90.998294,44.639539,14.786708,198.965365,15.287258,176.995433,...,-66.350467,111.417585,120.193970,4.299321,5.301497,89.034940,16.202143,0.000000,-10.121863,25.226697
732543,22.057560,21.290257,9.543309,-0.886233,20.983363,10.118855,2.767303,48.847752,3.304005,42.898281,...,-28.112983,24.256848,28.984773,2.170377,-0.069817,21.764476,1.841087,0.000000,-3.189972,4.833608
732544,-31.943424,-28.392806,-13.597494,0.195301,-27.257292,-13.583754,-5.220498,-55.643577,-4.855872,-49.526906,...,2.699866,-34.840293,-34.360279,-0.198994,-2.756257,-24.772632,-7.309493,0.000000,1.725870,-8.793818


In [4]:
eeg_data = eeg_data.to_numpy()
eeg_data.shape

(732546, 64)

In [2]:
feeltrace_mat = join(path_to_feeltrace, '2', 'feeltrace.mat')
feeltrace = loadmat(feeltrace_mat)
feeltrace = feeltrace['feeltrace']
timestamps = feeltrace[:,0]

In [6]:
p2_feeltrace = feeltrace
new_timestamp = np.arange(p2_feeltrace[0,0], p2_feeltrace[-1,0], 1000)
p2_feeltrace_even = np.interp(new_timestamp, p2_feeltrace[:,0], p2_feeltrace[:,1])
p2_feeltrace_slope = np.diff(p2_feeltrace_even)
np.save(join(path_to_prediction_data, '2', 'data_label.npy'), p2_feeltrace_slope)
np.save(join(path_to_prediction_data, '2', 'data_state.npy'), p2_feeltrace_even[:-1])

In [8]:
sfreq = 1000
num_channels = 64

tmin = int(timestamps[0])
tmax = int(timestamps[-1])

eeg = eeg_data[tmin:tmax, :]
num_second = eeg.shape[0] // sfreq
eeg = eeg[:num_second * sfreq, :]
eeg = eeg.reshape((num_second, sfreq, num_channels))

In [9]:
eeg.shape

(614, 1000, 64)

In [10]:
gamma_channels_index = [20, 25, 26, 27, 45]
gamma_channels_names = ["CP1(21)", "CP5(26)", "P5(27)", "P3(28)", "CP6(46)"]

beta_channels_index = [50, 31, 59, 25]
beta_channels_names = ["C2(51)", "P9(32)", "F7(60)", "CP5(26)"]

df_channels_index = [31, 26, 29, 30, 53]
df_channels_names = ["P9(32)", "P5(27)", "P7(30)", "P1(31)", "FC2(54)"]

https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/5165369/6933964/6858031/jenke1-2339834-large.gif

In [11]:
def compute_metric(X, metric, axis=None):
    if metric == "mean":
        return np.mean(X, axis=axis)
    elif metric == "min":
        return np.min(X, axis=axis)
    elif metric == "max":
        return np.max(X, axis=axis)
    elif metric == "var":
        return np.var(X, axis=axis)
        
def compute_gamma(X):
    # uncomment to run fft gamma band on all channels
#     gamma_channels_index = np.arange(64)
#     gamma_channels_names = ["E" + str(i+1) for i in gamma_channels_index]
    lo, hi = 30, 64
    metrics = ["mean", "min", "max", "var"]
    
    feature_names = []
    values = np.array(())
    X_fft = np.fft.fft(X[:, gamma_channels_index], axis=0)
    freqs = np.fft.fftfreq(sfreq, d=1/sfreq)
    X_gamma_fft = X_fft[np.logical_and(np.abs(freqs) > lo, np.abs(freqs) < hi)]
    
    for m in metrics:
        feature_names += map(lambda name: name + "_gamma_fft_" + m, gamma_channels_names)
        values = np.append(values, np.abs(compute_metric(X_gamma_fft, m, 0))) # append the magnitude of fft band
              
    return feature_names, values

def compute_beta(X):
    # uncomment to run fft gamma band with all channels
#     beta_channels_index = np.arange(64)
#     beta_channels_names = ["E" + str(i+1) for i in gamma_channels_index]
    lo, hi = 12, 30
    metrics = ["mean", "min", "max", "var"]
    
    feature_names = []
    values = np.array(())
    X_fft = np.fft.fft(X[:, beta_channels_index], axis=0)
    freqs = np.fft.fftfreq(sfreq, d=1/sfreq)
    X_beta_fft = X_fft[np.logical_and(np.abs(freqs) > lo, np.abs(freqs) < hi)]

    for m in metrics:
        feature_names += map(lambda name: name + "_beta_fft_" + m, beta_channels_names)
        values = np.append(values, np.abs(compute_metric(X_beta_fft, m, 0))) # append the magnitude of fft band
              
    return feature_names, values

def compute_first_difference(X):
    df_channels_index = [31, 26, 29, 30, 53]
    df_channels_names = ["P9(32)", "P5(27)", "P7(30)", "P1(31)", "FC2(54)"]
    feature_names = [cname + "_first_difference" for cname in df_channels_names]
    return feature_names, np.mean(np.abs(np.diff(X[:, df_channels_index], axis=0)), axis=0)

"""
4 freq band pairs: 
    beta-beta, alpha-gamma, beta-gamma, gamma-gamma
4 stats per pair:
    sum of the bispectrum magnitudes
    sum of the squares of the bispectrum magnitudes
    sum of the bicoherence magnitudes
    sum of the squares of the bicoherence magnitudes
2 channels:
    C2(51), FC2(54)
===> 4 * 4 * 2 = 32 features
"""
def compute_bicoherence_band(X):
    bic_channels_index = [50, 53]
    bic_channels_names = ["C2(51)", "FC2(54)"]
    metrics = ["sum_bis_magnitudes", "sum_of_sq_bis_magnitudes", "sum_bic_magnitudes", "sum_of_sq_bic_magnitudes"]
    band_pairs = ["beta-beta", "alpha-gamma", "beta-gamma", "gamma-gamma"]
    feature_names = []
    values = np.array(())
    
    def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = signal.butter(order, [low, high], btype='band')
        return b, a
    
    def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = signal.lfilter(b, a, data)
        return y
    
    for i in range(len(bic_channels_index)):
        sig = X[:, bic_channels_index[i]]
        sig_alpha = butter_bandpass_filter(sig, 8, 12, sfreq)
        sig_beta = butter_bandpass_filter(sig, 12, 30, sfreq)
        sig_gamma = butter_bandpass_filter(sig, 30, 64, sfreq)
        
        for pair in band_pairs:
            if pair == "beta-beta":
                f, bic, bis = compute_bicoherence(sig_beta, sig_beta, sfreq)
            elif pair == "alpha-gamma":
                f, bic, bis = compute_bicoherence(sig_alpha, sig_gamma, sfreq)
            elif pair == "beta-gamma":
                f, bic, bis = compute_bicoherence(sig_beta, sig_gamma, sfreq)
            elif pair == "gamma-gamma":
                f, bic, bis = compute_bicoherence(sig_gamma, sig_gamma, sfreq)
                
            feature_names += [bic_channels_names[i] + "_" + pair + "_" + m for m in metrics]
            values = np.append(values, np.sum(bis))
            values = np.append(values, np.sum(np.square(bis)))
            values = np.append(values, np.sum(bic))
            values = np.append(values, np.sum(np.square(bic)))
            
    return feature_names, values

def compute_bicoherence(s1, s2, rate, nperseg=sfreq, noverlap=sfreq/2):
    """ Compute the bicoherence between two signals of the same lengths s1 and s2
    using the function scipy.signal.spectrogram
    https://stackoverflow.com/questions/4194554/function-for-computing-bicoherence
    """
    # compute the stft
    f1, t1, spec_s1 = signal.spectrogram(s1, fs = rate, nperseg = nperseg, noverlap = noverlap, mode = 'complex',)
    f2, t2, spec_s2 = signal.spectrogram(s2, fs = rate, nperseg = nperseg, noverlap = noverlap, mode = 'complex')

    # transpose (f, t) -> (t, f)
    spec_s1 = np.transpose(spec_s1, [1, 0])
    spec_s2 = np.transpose(spec_s2, [1, 0])

    # compute the bicoherence
    arg = np.arange(f1.size / 2).astype('int')
    sumarg = arg[:, None] + arg[None, :]
    bis = np.abs(
        np.mean(spec_s1[:, arg, None] * spec_s1[:, None, arg] * np.conjugate(spec_s2[:, sumarg]), 
        axis = 0)
        ) ** 2
    denum = np.mean(
        np.abs(spec_s1[:, arg, None] * spec_s1[:, None, arg]) ** 2, axis = 0) * np.mean(
            np.abs(np.conjugate(spec_s2[:, sumarg])) ** 2, 
            axis = 0)
    bic = bis / denum
    print(bic[:4, :4])
    return f1[arg], bic, bis

def compute_hjorth(X):
    hjorth_channels_index = [20, 26, 30, 40]
    hjorth_channels_names = ["CP1(21)", "P5(27)", "P1(31)", "CP2(41)"]
    metrics = ["mobility", "complexity"]
    
    data = X[:, hjorth_channels_index].T
    
    feature_names = [i + "_hjorth_" + metrics[0] for i in hjorth_channels_names] + [i + "_hjorth_" + metrics[1] for i in hjorth_channels_names]
    values = np.append(mne_features.univariate.compute_hjorth_mobility(data), mne_features.univariate.compute_hjorth_complexity(data))
    
    return feature_names, values

def compute_fractal_dimension(X):
    fd_channels_index = [20, 26, 30, 40]
    fd_channels_names = ["CP1(21)", "P5(27)", "P1(31)", "CP2(41)"]
    
    data = X[:, fd_channels_index].T
    feature_names = [i + "_fractal_dimension" for i in fd_channels_names]
    return feature_names, mne_features.univariate.compute_higuchi_fd(data)

def compute_HOC(X, n=10):
    HOC_channels_index = [20, 40, 51, 42]
    HOC_channels_names = ["CP1(21)", "CP2(41)", "T8(52)", "P10(43)"]
    
    data = X[:, HOC_channels_index].T
    feature_names = []
    values = np.array(())
    for i in range(n):
        feature_names += [channel_name + "_" + str(i) + "_order_crossing" for channel_name in HOC_channels_names]
        values = np.append(values, mne_features.univariate.compute_zero_crossings(data))
        data = np.diff(data)
        
    return feature_names, values

In [12]:
def compute_features(X):
    # X.shape -> (sfreq, num_channels)
    # res.shape -> (sfreq, num_features)
    
    feature_names = []
    result = np.array(())
    
    names, values = compute_gamma(X)
    feature_names += names
    result = np.append(result, values)
    
    names, values = compute_beta(X)
    feature_names += names
    result = np.append(result, values)
    
    names, values = compute_first_difference(X)
    feature_names += names
    result = np.append(result, values)
    
#     names, values = compute_bicoherence_band(X)
#     feature_names += names
#     result = np.append(result, values)
    
    names, values = compute_fractal_dimension(X)
    feature_names += names
    result = np.append(result, values)
    
    names, values = compute_hjorth(X)
    feature_names += names
    result = np.append(result, values)
    
    names, values = compute_HOC(X)
    feature_names += names
    result = np.append(result, values)
    
    return feature_names, result

def compute_features_wrap(X):
    feature_names, result = compute_features(X)
    return result

In [13]:
X = eeg[0]
feature_names, result = compute_features(X)

In [14]:
X = np.array(list(map(compute_features_wrap, eeg)))

In [15]:
X.shape

(614, 93)

In [20]:
len(feature_names)

93

In [119]:
np.save(join(path_to_prediction_data, '2', 'data_features.npy'), X)